In [76]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import xgboost as xgb
from tqdm import tqdm, tqdm_pandas
tqdm_pandas(tqdm())
import datetime
from sklearn.cross_validation import train_test_split
import operator


0it [00:00, ?it/s]


In [95]:
train = pd.read_csv('./train_1.csv').fillna(0)

In [2]:
key = pd.read_csv('./key_trunc.csv')

In [3]:
sub = pd.read_csv("./sample_submission_1.csv")

In [2]:
train_melt = pd.read_csv("train_feats.csv")

In [3]:
for col in train.columns[1:]:
    train[col] = pd.to_numeric(train[col],downcast='integer')

NameError: name 'train' is not defined

In [3]:
def get_median(row, last_n_days):
    return pd.rolling_median(pd.Series(row), window = last_n_days)    

In [4]:
def get_mean(row, last_n_days):
    return pd.rolling_mean(pd.Series(row), window = last_n_days)    

In [121]:
def get_mean_diff(row):
    return np.mean(np.diff(pd.Series(row)))

In [130]:
def get_long_stats(func_type, df, df_melt, last_n_days = None):

    if func_type == "median":
        rolling_stats = df.iloc[:, -60:].progress_apply(get_median, last_n_days = last_n_days, axis = 1)
    elif func_type == "mean":
        rolling_stats = df.iloc[:, -60:].progress_apply(get_mean, last_n_days = last_n_days, axis = 1)
        
    rolling_stats_df = pd.concat([df.Page, rolling_stats.apply(pd.Series)], axis = 1)    
    rolling_stats_df.columns = ["Page"] + list(df.columns[-60:])
    
    rolling_stats_melt = pd.melt(rolling_stats_df, id_vars=['Page'], var_name="date", value_name=func_type + "_" + str(last_n_days))
    
    df_melt = pd.merge(df_melt, rolling_stats_melt, on = ["Page", "date"])     
    return df_melt

In [ ]:
def generate_train_melt():

#     train_melt = pd.read_csv("train_feats.csv")

#     Commented because these features are already generated
    train_melt = pd.melt(pd.concat([train.Page, train.iloc[:, -60:]], axis = 1), id_vars=['Page'], var_name="date", value_name="visits")

    train_melt = get_long_stats("mean", train, train_melt, 7)
    train_melt = get_long_stats("mean", train, train_melt, 30)
    train_melt = get_long_stats("mean", train, train_melt, 60)

    train_melt = get_long_stats("median", train, train_melt, 7)
    train_melt = get_long_stats("median", train, train_melt, 30)
    train_melt = get_long_stats("median", train, train_melt, 60)
    
    train_melt["date_num"] = train_melt.progress_apply(get_date, axis = 1)
    train_melt["weekday"] = train_melt.progress_apply(get_weekday, axis = 1)
    
    train_melt["mean_diff"] = train.iloc[:, -7:].progress_apply(get_mean_diff, axis = 1)
    train_melt["last_day_visits"] = train.iloc[:, -1:]
    
    #Columns i am not using
    #train_melt["month"] = train_melt.progress_apply(get_month, axis = 1)
    # train_melt["language"] = train_melt.progress_apply(get_language, axis = 1)
    
    return train_melt 

train_melt = generate_train_melt()
train_melt.to_csv("train_feats.csv", index = False)


  0%|          | 0/145063 [00:00<?, ?it/s]/Library/Python/2.7/site-packages/ipykernel_launcher.py:2: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=7,center=False).mean()
  

 26%|██▌       | 37283/145063 [00:13<00:44, 2445.20it/s]


 53%|█████▎    | 76721/145063 [00:28<00:23, 2961.48it/s]


 79%|███████▉  | 115122/145063 [00:42<00:10, 2781.62it/s]


100%|█████████▉| 144876/145063 [00:53<00:00, 2832.22it/s]
145064it [00:56, 2575.61it/s]                            
  0%|          | 0/145063 [00:00<?, ?it/s]/Library/Python/2.7/site-packages/ipykernel_launcher.py:2: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=30,center=False).mean()
  

  5%|▍         | 6844/145063 [00:02<00:48, 2861.58it/s]


 32%|███▏      | 46112/145063 [00:16<00:35, 2820.43it/s]


 39%|███▉      | 56226/145063 [00:20<00:31, 2860.49it/s]

In [7]:
def get_source(row):
    return row.Page.split["."][-1].split("_")[-1]

In [8]:
def get_language(row):
    try:
        return row.Page.split(".")[0].split("_")[-1]    
    except:
        return "en"

In [9]:
def get_month(row):
    return row.date.split("-")[1]

In [10]:
def get_date(row):
    return row.date.split("-")[2]

In [11]:
def get_weekday(row):
    return datetime.datetime.strptime(row.date, '%Y-%m-%d').date().weekday()

In [13]:
# def get_language_mean(row):
# lang_count = pd.Series(train_melt.groupby(["language"])["Page"].count()).sort_values(axis = 0, ascending = False)
# print lang_count

In [12]:
def get_median_7(row, last_index):
    
    return np.median(row.iloc[last_index - 7 : last_index].tolist())

In [13]:
def get_median_30(row, last_index):
    
    return np.median(row.iloc[last_index - 30 : last_index].tolist())

In [14]:
def get_median_60(row, last_index):
    
    return np.median(row.iloc[last_index - 60 : last_index].tolist())

In [15]:
def get_mean_7(row, last_index):
    
    return np.mean(row.iloc[last_index - 7 : last_index].tolist())

In [16]:
def get_mean_30(row, last_index):
    
    return np.mean(row.iloc[last_index - 30 : last_index].tolist())

In [17]:
def get_mean_60(row, last_index):
    
    return np.mean(row.iloc[last_index - 60 : last_index].tolist())

In [18]:
# def get_test_feats(row, last_index):
    
#     subset_row = row.iloc[last_index - 60 : last_index].tolist()
    
#     median_7 = np.median(subset_row[-7:])
#     median_30 = np.median(subset_row[-30:])
#     median_60 = np.median(subset_row[-60:])

#     mean_7 = np.mean(subset_row[-7:])
#     mean_30 = np.mean(subset_row[-30:])
#     mean_60 = np.mean(subset_row[-60:])
    
#     return pd.Series([row.Page, median_7, median_30, median_60, mean_7, mean_30, mean_60])

In [66]:
def smape(y_pred, y_true):
        
    y_true = y_true.get_label()
    y_pred = np.array(y_pred)
        
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 200.0
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0

    return "smape", np.nanmean(diff)

In [90]:
def run_xgb(train, label_train, valid = None, label_valid = None):

    # Set our parameters for xgboost
    params = {}
    params['objective'] = 'reg:linear'
    params['eta'] = 0.1
    params['max_depth'] = 6
    params['silent'] = 1
    params['min_child_weight'] = 0
    params['subsample'] = 1
    params['colsample_bytree'] = 1
    params['nthread'] = 13
    params['gamma'] = 0
    params['max_delta_step'] = 0

    d_train = xgb.DMatrix(train, label=label_train)
    
    if valid is not None:
        d_valid = xgb.DMatrix(valid, label=label_valid)
        watchlist = [(d_train, 'train'), (d_valid, 'validation')]
    else:
        watchlist = [(d_train, 'train')]
        
    bst = xgb.train(params, d_train, 200, watchlist, early_stopping_rounds=10, verbose_eval=10, feval=smape, maximize = False)
    
    return bst

In [23]:
label_train_melt = train_melt["visits"]

train_melt.drop("visits", axis = 1, inplace = True)
train_melt.drop("date", axis = 1, inplace = True)
train_melt.drop("month", axis = 1, inplace = True)
train_melt.drop("language", axis = 1, inplace = True)
train_melt.drop("Page", axis = 1, inplace = True)

In [91]:
x_train, x_valid, label_train, label_valid = train_test_split(train_melt, label_train_melt, test_size=0.2, random_state=4242)

bst = run_xgb(x_train, label_train, x_valid, label_valid)
# bst = run_xgb(train_melt, label_train_melt)

[0]	train-rmse:81076.6	validation-rmse:81130.6	train-smape:143.502	validation-smape:142.575
Multiple eval metrics have been passed: 'validation-smape' will be used for early stopping.

Will train until validation-smape hasn't improved in 10 rounds.
[10]	train-rmse:80501.6	validation-rmse:80922.4	train-smape:65.3812	validation-smape:65.9836
[20]	train-rmse:80344.4	validation-rmse:80898.1	train-smape:54.8142	validation-smape:55.2881
[30]	train-rmse:80291	validation-rmse:80899	train-smape:51.2666	validation-smape:51.7054
[40]	train-rmse:80271	validation-rmse:80903.5	train-smape:52.7021	validation-smape:53.5681
Stopping. Best iteration:
[33]	train-rmse:80282.4	validation-rmse:80900.6	train-smape:50.7296	validation-smape:51.2577



In [92]:
importance = bst.get_fscore()

importance = sorted(importance.items(), key=operator.itemgetter(1))

df_imp = pd.DataFrame(importance, columns=['feature', 'fscore'])
df_imp['fscore'] = df_imp['fscore']/df_imp['fscore'].sum()

print df_imp

     feature    fscore
0    mean_60  0.008529
1  median_60  0.009950
2    weekday  0.114428
3    mean_30  0.120824
4   date_num  0.125089
5  median_30  0.130775
6   median_7  0.235963
7     mean_7  0.254442


In [32]:
def get_preds(test_feats):
    
    d_test = xgb.DMatrix(test_feats)
    p_test = bst.predict(d_test)
    
    return pd.Series(p_test)

In [24]:
def get_overall_df():
    
#     key = pd.read_csv("key_trunc.csv")
    key_wide = pd.DataFrame()
    key_wide["page"] = np.unique(key.trunc_page).tolist()
    for i in np.unique(key.date).tolist():
        key_wide[i] = 0

    overall_df = pd.merge(train, key_wide, left_on="Page", right_on = "page", how = "inner")
    overall_df.drop("page", axis = 1, inplace = True)
    
    return overall_df

In [40]:
def test_and_pred(overall_df):
    
    pred_df = pd.DataFrame()
    
    for i in range(overall_df.columns.tolist().index('2017-01-01'), len(overall_df.columns)):

        last_date = overall_df.columns.tolist()[i]
        print last_date

        last_index = overall_df.columns.tolist().index(last_date)

        test_feats = pd.DataFrame()
        
        test_feats["Page"] = overall_df.Page
        test_feats["median_7"] = overall_df.apply(get_median_7, axis = 1, last_index = last_index)
        test_feats["median_30"] = overall_df.apply(get_median_30, axis = 1, last_index = last_index)
        test_feats["median_60"] = overall_df.apply(get_median_60, axis = 1, last_index = last_index)
        test_feats["mean_7"] = overall_df.apply(get_mean_7, axis = 1, last_index = last_index)
        test_feats["mean_30"] = overall_df.apply(get_mean_30, axis = 1, last_index = last_index)
        test_feats["mean_60"] = overall_df.apply(get_mean_60, axis = 1, last_index = last_index)

        test_feats["month"] = last_date.split("-")[1]
        test_feats["datenum"] = last_date.split("-")[2]
        test_feats["weekday"] = datetime.datetime.strptime(last_date, '%Y-%m-%d').date().weekday()
        
        prediction = get_preds(test_feats)

        overall_df[last_date] = prediction
        pred_df = pd.concat([pred_df, pd.concat([overall_df.Page, prediction], axis = 1)], axis = 1)
        
    return overall_df

In [41]:
# overall_df = get_overall_df()
overall_df = test_and_pred(overall_df)

2017-01-01
                                                Page  median_7  median_30  \
0            2NE1_zh.wikipedia.org_all-access_spider      20.0       19.5   
1             2PM_zh.wikipedia.org_all-access_spider      26.0       22.0   
2              3C_zh.wikipedia.org_all-access_spider       4.0        3.5   
3         4minute_zh.wikipedia.org_all-access_spider      16.0       16.5   
4  52_Hz_I_Love_You_zh.wikipedia.org_all-access_s...      13.0        8.5   

   median_60     mean_7    mean_30    mean_60 month datenum  weekday  
0       18.0  19.857143  24.966667  25.716667    01      01        6  
1       18.5  28.142857  25.000000  31.433333    01      01        6  
2        4.0   6.428571  11.133333   7.650000    01      01        6  
3       13.0  15.857143  17.400000  15.433333    01      01        6  
4       10.0  16.142857  11.933333  15.400000    01      01        6  
2017-01-02


KeyboardInterrupt: 

In [33]:
overall_df.head()

,Page,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,...,2017-02-20,2017-02-21,2017-02-22,2017-02-23,2017-02-24,2017-02-25,2017-02-26,2017-02-27,2017-02-28,2017-03-01
0,2NE1_zh.wikipedia.org_all-access_spider,18.0,11.0,5.0,13.0,14.0,9.0,9.0,22.0,26.0,...,0,0,0,0,0,0,0,0,0,0
1,2PM_zh.wikipedia.org_all-access_spider,11.0,14.0,15.0,18.0,11.0,13.0,22.0,11.0,10.0,...,0,0,0,0,0,0,0,0,0,0
2,3C_zh.wikipedia.org_all-access_spider,1.0,0.0,1.0,1.0,0.0,4.0,0.0,3.0,4.0,...,0,0,0,0,0,0,0,0,0,0
3,4minute_zh.wikipedia.org_all-access_spider,35.0,13.0,10.0,94.0,4.0,26.0,14.0,9.0,11.0,...,0,0,0,0,0,0,0,0,0,0
4,52_Hz_I_Love_You_zh.wikipedia.org_all-access_s...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [127]:
train_melt = pd.read_csv("train_feats.csv")
train_melt.head()

,mean_7,mean_30,mean_60,median_7,median_30,median_60,date_num,weekday
0,NaN,NaN,NaN,NaN,NaN,NaN,2,2
1,NaN,NaN,NaN,NaN,NaN,NaN,2,2
2,NaN,NaN,NaN,NaN,NaN,NaN,2,2
3,NaN,NaN,NaN,NaN,NaN,NaN,2,2
4,NaN,NaN,NaN,NaN,NaN,NaN,2,2


In [52]:
len(train_melt)

8703780

In [49]:
sub.head()

,Id,Visits
0,bf4edcf969af,0
1,929ed2bf52b9,0
2,ff29d0f51d5c,0
3,e98873359be6,0
4,fa012434263a,0


In [ ]:
# train_melt = pd.read_csv("train_feats.csv")
key = pd.read_csv("key_trunc.csv")

train_melt = pd.merge(train_melt, key, left_on="Page", right_on="trunc_page", how="inner")
train_melt.drop("Page_x", inplace=True, axis = 1)
train_melt.drop("Page_y", inplace=True, axis = 1)

train_melt.to_csv("train_merged.csv", index = False)

In [93]:
train_merged_filtered = train_merged[["Id", "median_60_days"]]

In [111]:
sub = pd.merge(sub, train_merged_filtered, on = "Id")
sub.drop("Visits", inplace=True, axis = 1)
sub.columns = ["Id", "Visits"]

In [112]:
sub.to_csv("median_60_days.csv", index = False)

In [50]:
key.head()

,Id,trunc_page,date
0,bf4edcf969af,!vote_en.wikipedia.org_all-access_all-agents,2017-01-01
1,929ed2bf52b9,!vote_en.wikipedia.org_all-access_all-agents,2017-01-02
2,ff29d0f51d5c,!vote_en.wikipedia.org_all-access_all-agents,2017-01-03
3,e98873359be6,!vote_en.wikipedia.org_all-access_all-agents,2017-01-04
4,fa012434263a,!vote_en.wikipedia.org_all-access_all-agents,2017-01-05
